# Diagnostics, analysis and visualization tools <br /> for Integrated Assessment timeseries data

<img style="float: right; height: 100px; margin-top: 10px;" src="_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="_static/IAMC_logo.jpg">

## First steps with the ``pyam_analysis`` package

The ``pyam-analysis`` package provides a range of diagnostic tools and functions  
for analyzing and working with IAMC-style timeseries data. 

The package can be used with data that follows the data template convention 
of the [Integrated Assessment Modeling Consortium](http://www.globalchange.umd.edu/iamc/) (IAMC).
An illustrative example is shown below;
see [data.ene.iiasa.ac.at/database](http://data.ene.iiasa.ac.at/database/) for more information.


| **Model**           | **Scenario**  | **Region** | **Variable**   | **Unit** | **2005** | **2010** | **2015** |
|---------------------|---------------|------------|----------------|----------|----------|----------|----------|
| MESSAGE V.4         | AMPERE3-Base  | World      | Primary Energy | EJ/y     | 454.5    |	479.6    | ...      |
| ...                 | ...           | ...        | ...            | ...      | ...      | ...      | ...      |

This notebook illustrates some basic functionality of the ``pyam-analsysis`` package
and the ``IamDataFrame`` class:

0. Importing timeseries data from a csv file.
0. Listing models, scenarios and variables included in the data.
0. Display of timeseries data as dataframe and visualization using simple plotting functions.
0. Evaluating the model data and executing a range of diagnostic checks to identify data outliers.
0. Categorization of scenarios according to timeseries data.
0. Exporting data to Excel


## Tutorial data

The timeseries data used in this tutorial is a partial snapshot of the scenario database 
compiled for the IPCC's Fifth Assessment Report (AR5):

> Krey V., O. Masera, G. Blanford, T. Bruckner, R. Cooke, K. Fisher-Vanden, H. Haberl, E. Hertwich, E. Kriegler, D. Mueller, S. Paltsev, L. Price, S. Schlömer, D. Ürge-Vorsatz, D. van Vuuren, and T. Zwickel, 2014: *Annex II: Metrics & Methodology*.   
> In: *Climate Change 2014: Mitigation of Climate Change. Contribution of Working Group III to the Fifth Assessment Report of the Intergovernmental Panel on Climate Change* [Edenhofer, O., R. Pichs-Madruga, Y. Sokona, E. Farahani, S. Kadner, K. Seyboth, A. Adler, I. Baum, S. Brunner, P. Eickemeier, B. Kriemann, J. Savolainen, S. Schlömer, C. von Stechow, T. Zwickel and J.C. Minx (eds.)]. Cambridge University Press, Cambridge, United Kingdom and New York, NY, USA. [Link](https://www.ipcc.ch/report/ar5/wg3/)

The complete database is publicly available at [tntcat.iiasa.ac.at/AR5DB/](https://tntcat.iiasa.ac.at/AR5DB/).

<img style="float: right; height: 100px;" src="_static/AMPERE-Logo.png">
<img style="float: right; height: 40px; margin-top: 50px; margin-right: 20px;" src="_static/EMF-Logo_v2.1.png">

The data snapshot used for this tutorial consists of selected data from two model intercomparison projects:
 - Energy Modeling Forum Round 27 
   ([**EMF27**](https://emf.stanford.edu/projects/emf-27-global-model-comparison-exercise)),
   see the Special Issue in [*Climatic Change* 3-4, 2014](https://link.springer.com/journal/10584/123/3/page/1).
 - EU FP7 project [**AMPERE**](https://tntcat.iiasa.ac.at/AMPEREDB/), 
   see the following scientific publications:
   > - Riahi, K., et al. (2015). "Locked into Copenhagen pledges — Implications of short-term emission targets 
   >   for the cost and feasibility of long-term climate goals." 
   >   *Technological Forecasting and Social Change* 90(Part A): 8-23.  
   >   [DOI: 10.1016/j.techfore.2013.09.016](https://doi.org/10.1016/j.techfore.2013.09.016)
   > - Kriegler, E., et al. (2015). "Making or breaking climate targets: The AMPERE study on 
   >   staged accession scenarios for climate policy."
   >   *Technological Forecasting and Social Change* 90(Part A): 24-44.  
   >   [DOI: 10.1016/j.techfore.2013.09.021](https://doi.org/10.1016/j.techfore.2013.09.021)

<div style="text-align: center; padding: 10px; border: 2px solid red; width: 700px">
*The data used in this tutorial is ONLY a partial snapshot of the IPCC AR5 scenario database!*  
*This tutorial is only intended for an illustration of the ``pyam-analysis`` package.*
</div>

## Import package and load data from the AR5 tutorial csv snapshot file

First, we import the snapshot timeseries data from the file ``tutorial_AR5_data.csv`` in the ``tutorial`` folder.

As a first step, we show lists of all models, scenarios, regions, and variables (with units) included in the snapshot.

In [ ]:
import pyam_analysis as iam
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = iam.IamDataFrame(data='tutorial_AR5_data.csv', encoding='utf-8')

In [ ]:
df['model'].unique()

In [ ]:
df[['model', 'scenario']].head()

In [ ]:
df[['variable', 'unit']].head()

## Tutorial on data filtering
Most functions of the ``IamDataFrame`` class take an (optional) argument ``filters``,
i.e., a dictionary of filter criteria.

### Filtering by model names, scenarios and regions

The feature for filtering by **model, scenario or region** is implemented
using [regular expressions](https://docs.python.org/3.6/library/re.html) (regex, re)
and the _re.match()_ function.
This implies that the filtering is done from the beginning of the text string.
> Applying the filter ``'model': 'MESSAGE'`` to the function ``scenarios()`` 
  will return all MESSAGE V.4 scenarios included in the snapshot.  
> Filtering for _ESSAGE_ will return an empty set.

In [ ]:
df.filter({'model': 'MESSAGE'})['scenario'].unique()

In [ ]:
df.filter({'model': 'ESSAGE'})['scenario'].unique()

### Filtering by variables and hierarchy levels

Filtering for **variable** strings using regex
is problematic due to the frequent use of the ``"|"`` character in the IAMC template
to specify a hierarchical.
Therefore, this package implements a pseudo-regex syntax, 
where ``|`` is escaped, ``*`` is used as a wildcard
and exact matching at the end of the string is enforced.
(in regex lingo, ``*`` is replaced by ``.*`` and ``$`` is appended to the filter string).
> Filtering for _Primary Energy_ will return only exactly those data.  
> Filtering for _Primary Energy|*_ will return all sub-categories of 
> primary-energy level (and only the sub-categories).

In additon, IAM variables can be filtered by the **level**,
i.e., the "depth" of the variable in a hierarchical reading of the string separated by `"|"`.
That is, the variable _Primary Energy_ has level 0, while _Primary Energy|Coal_ has level 1.
Filtering by both **variables** and **level** will search for the hierarchical depth 
_following the <variable> string_, so filter arguments _Primary Energy|*_ and _level = 0_
will return all variables immediately below _Primary Energy_.
Filtering by **level** only will return all variables up to that depth.

To illustrate the functionality of the filters, we first show all sub-categories of the ``Emissions`` variable.  
Then, we reduce variables to only two hierarchical levels below ``"Emissions|"``; the list returned by the function call will not include ``Emissions|CO2|Fossil Fuels and Industry|Energy Supply|Electricity``, because this variable is three hierarchical levels below ``"Emissions|"``.

The third example shows how to filter only by hierarchical level. The function returns all variables that are at the top hierarchical level (i.e., ``Primary Energy``) and those at the first sub-category level.
Keep in mind that there are no variables ``Emissions`` or ``Price`` (no top level).

In [ ]:
df.filter(filters={'variable': 'Emissions|*'})['variable'].unique()

In [ ]:
df.filter(filters={'variable': 'Emissions|*', 'level': 2})['variable'].unique()

In [ ]:
df.filter(filters={'level': 1})['variable'].unique()

### Filtering by year

Filtering for **years** can be done by integer number, a list of integers, or the Python class ``range``.  
Note that the last year of a range is not included, so ``range(2010,2015)``
is interpreted as ``[2010, 2011, 2012, 2013, 2014]``.

## Getting help

When in doubt, you can look at the help for any function by appending it with a ``?``.

In [ ]:
df.filter?

## Displaying timeseries data and plotting figures

As a next step, we want to view a selection of the data in the tutorial snapshot using the IAMC standard.
The filtered data can exported as a csv file by appending `.to_csv('selected_data.csv')` to the next command.

For displaying data in a different format, the class ``IamDataFrame`` has a wrapper of the ``pandas.DataFrame.pivot_table()`` function. It allows to flexibly specify the columns and rows.
The function automatically aggregates by summation or counting (specified by the parameter `aggfunc`) 
over all timeseries data identifiers ('model', 'scenario', 'variable', 'region', 'unit', 'year')
which are not used as `index` or `columns`.

In the example below, the filter of the timeseries data is set for all subcategories of 'Primary Energy', 
which are then summed up in the displayed table.

In [ ]:
(df
 .filter(filters={'scenario': 'AMPERE3-450', 'variable': 'Primary Energy|Coal', 'region': 'World'})
 .timeseries()
)

In [ ]:
(df
 .filter({'variable': 'Primary Energy', 'region': 'World'})
 .pivot_table(index=['year'], columns=['scenario'], values='value', aggfunc='sum')
)

If you are familiar with the `python` package `pandas`, you can treat the IamDataFrame similarly.

In [ ]:
df.head()

## Visualization of timeseries

Please see the `plotting.ipynb` notebook for a full tutorial on plotting.

## Validation and diagnostic assessment of timeseries data

When analyzing scenario results, it is often useful to check whether certain timeseries exist or the values are within a specific range. For example, it may make sense to ensure that reported data for historical periods are close to established reference data.

The following section provides three illustrations:
1. Check whether a timeseries `'Primary Energy'` exists in each scenario (in at least one year).
2. Check for every scenario whether the value for `'Primary Energy'` at the global level exceeds 515 EJ/y 
   in the reference year 2010
   (the value must satisfy an upper bound of 515 EJ/y in this notation).
3. Check for every scenario whether the value for `'Primary Energy|Coal'` exceeds 400 EJ/y in mid-century.

The `validate()` function takes a `filters` dictionary to perform the checks on a selection of models/scenarios
similar to the functions introduced above.  
The ``criteria`` argument can specify a valid range by an upper and lower bound (``up``, ``lo``) for a variable and a subset of years to which the validation is applied - all scenarios with a value in at least one year outside that range are considered to *not satisfy* the validation.

By setting the argument ``exclude=True``, all scenarios failing the validation will be categorized as ``exclude``.
These scenarios will not be shown by default in any subsequent data tables or plots.

In [ ]:
df.validate?

In [ ]:
df.validate(criteria={'Primary Energy': {'up': 515, 'year': 2010}})

In [ ]:
(df
 .filter(filters={'region': 'World'})
 .validate(criteria={'Primary Energy|Coal': {'up': 400, 'year': 2050}})
)

In [ ]:
iam.validate(df, 
             filters={'region': 'World'}, 
             criteria={'Primary Energy|Coal': {'up': 400, 'year': 2050}}
)

## Categorization of scenarios by timeseries characteristics

It is often useful to apply categorization to classes of scenarios according to specific characteristics of the timeseries data.

In the following example, we use the temperature change assessment by MAGICC 6 to group scenarios by the median global warming by the end of the century (year 2100).

We proceed in the following steps:

0. Plot the timeseries data of the variable that we want to use. 
   This provides some insights on useful thresholds for the categorization.
0. Use the function ``category()`` to apply a categorization (and colour code for later use) 
   to all scenarios that satisfy a number of specific criteria.
0. Use the categorization of scenarios for analysis of other timeseries data.

In [ ]:
df.filter({'variable': 'Temperature*'}).line_plot()

We now use the categorization feature of the ``pyam-analysis`` package. 
By default, each model/scenario is assigned as ``"uncategorized"``.

The next function resets all scenarios back to ``"uncategorized"``. This may be helpful in this tutorial if you are going back and forth between cells.

In [ ]:
df.categorize(
    'Temperature', 'Below 1.6C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 1.6, 'year': 2100}},
    color='cornflowerblue'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.0C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 2.0, 'lo': 1.6, 'year': 2100}},
    color='forestgreen'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 2.5C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 2.5, 'lo': 2.0, 'year': 2100}},
    color='gold'
)

In [ ]:
df.categorize(
    'Temperature', 'Below 3.5C',
     criteria={'Temperature|Global Mean|MAGICC6|MED': {'up': 3.5, 'lo': 2.5, 'year': 2100}},
     color='firebrick'
)

In [ ]:
df.categorize(
    'Temperature', 'Above 3.5C',
    criteria={'Temperature|Global Mean|MAGICC6|MED': {'lo': 3.5, 'year': 2100}},
    color='magenta'
)

In [ ]:
df.categorize('Temperature', 'No Data', criteria='uncategorized', color='white')

Two models included in the snapshot have not been assessed by MAGICC6 regarding their long-term climate and warming impact. Therefore, the timeseries ``'Temperature|Global Mean|MAGICC6|MED'`` does not exist, and they have not been categorized.

Below, we display all scenarios that are uncategorized at this point.

In [ ]:
df['Temperature'].head(n=10)

Now, we again display the median global temperature increase for all scenarios, but we use the colouring by category to illustrate the common charateristics across scenarios.

In [ ]:
df.filter({'variable': 'Temperature*'}).line_plot(color='Temperature', legend=True)

As a last step, we display the aggregate CO2 emissions by category. This allows to highlight alternative pathways within the same category. 

In this step, we also export this figure as a png using the option ``save``. The figure will be saved in the tutorials folder.

In [ ]:
fig, ax = plt.subplots()
(df
 .filter({'variable': 'Emissions|CO2', 'region': 'World'})
 .line_plot(ax=ax, color='Temperature', legend=True)
)
fig.savefig('co2_emissions.png')

## Writing data to Excel

The IamDataFrame can be directly exported to Excel in the standard IAMC format.
This feature is based on [pandas.DataFrame.to_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html) and can use any keyword arguments of that function.

In [ ]:
df.to_excel('tutorial_export.xlsx')
df.export_metadata('tutorial_metadata.xlsx')